# Project Notes
this is about saving the amazon rainforest

here we try to improve level1 results (0.899) by
- input data normalization and some clean up
- data augmentation

ressources for replacing the deprecated image data generator module:
- https://www.tensorflow.org/tutorials/load_data/images
- https://www.tensorflow.org/guide/keras/preprocessing_layers
- https://www.tensorflow.org/tutorials/images/data_augmentation

## result
- val_loss: 0.1050 - val_binary_accuracy: 0.9592
- private score: 0.906
- public score: 0.909

# 1.Import dependencies

In [1]:
import sys
import os
import subprocess

from six import string_types

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display

import random
from time import time
from glob import glob

# from tqdm.notebook import tqdm
from collections import Counter
# import dill as pickle

from plotly import graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots




import cv2

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score, confusion_matrix

# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, Dataset
# from torchvision import transforms as T, models
# from torch.optim import Adam
# from torch.optim.lr_scheduler import StepLR
# !pip install -q torchsummary --user
# from torchsummary import summary

# 2.Load Data

In [2]:
PLANET_KAGGLE_LEVEL_ROOT = os.path.abspath("C:/Users/janni/RomansCode/Rainforest ML Projekt")
PLANET_KAGGLE_ROOT = os.path.abspath("C:/Users/janni/RomansCode/Rainforest ML Projekt/input")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_JPG_TEST_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'test-jpg')
PLANET_KAGGLE_JPG_TEST_EXTRA_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'test-jpg-additional')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
PLANET_KAGGLE_SUBMISSION_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'sample_submission_orig.csv')

assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)
assert os.path.exists(PLANET_KAGGLE_JPG_TEST_DIR)
assert os.path.exists(PLANET_KAGGLE_JPG_TEST_EXTRA_DIR)

assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)

assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

assert os.path.exists(PLANET_KAGGLE_JPG_TEST_DIR)
assert os.path.exists(PLANET_KAGGLE_JPG_TEST_EXTRA_DIR)


In [3]:
#parameters
exec_train = False
exec_load = True
exec_test = False
exec_submit = False
exec_val = True
exec_visualize = False

use_best_thresholds = True
manual_threshs = [0.2]*17 #only necessary if you dont use the best thresholds

In [4]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [5]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [6]:
# Add onehot features for every label
labels_df_original = labels_df.copy()
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
labels_df.head()

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


# get a simple one working
https://www.kaggle.com/code/emekeh/planet-dataset-stage-d

In [7]:
# Importing the necessary libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import gc 
import matplotlib.pyplot as plt
import cv2 
import tensorflow as tf
# from tensorflow import keras
#from tqdm import tqdm 
from matplotlib.image import imread
from sklearn.model_selection import train_test_split
from keras import optimizers
# from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping 
from keras.preprocessing.image import ImageDataGenerator

# from tensorflow.keras.optimizers import Adam

from keras.optimizers import Adam

C:\Users\janni\anaconda3\envs\ml_python3_7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\janni\anaconda3\envs\ml_python3_7\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\janni\anaconda3\envs\ml_python3_7\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\janni\anaconda3\envs\ml_python3_7\lib\site-

In [8]:
import warnings
# filter warnings
warnings.filterwarnings('ignore')

In [9]:
gc.collect() #Frequently used to avoid session crashing due to memory exhaustion

22

In [10]:
# Initializing imagedatagenerator with a validation split of 0.2
train_datagen = ImageDataGenerator(rescale = 1./255., validation_split = 0.2, featurewise_std_normalization=True, horizontal_flip=True, vertical_flip=True, featurewise_center=True)

#we convert to dataframe used up to this point so that it fits into this guy's architecture

# adding .jpg extension to the column image_name so as to have same name format as the image files
train_df1 = labels_df.copy()
train_df1['image_name'] = labels_df['image_name'].apply(lambda x: '{}.jpg'.format(x))
# train_df1['image_name'] = labels_df['image_name'].apply(lambda x: '{}.tif'.format(x))
train_df1.head()



,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0.jpg,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1.jpg,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2.jpg,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3.jpg,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [11]:
columns = list(train_df1.columns[2:])
columns

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [12]:
# # Generating train data generator 
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df1,
                                                    directory =PLANET_KAGGLE_JPEG_DIR, 
                                                    x_col='image_name', y_col=columns, subset='training', 
                                                    batch_size=32,seed=42, shuffle=True, 
                                                    class_mode='raw', target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_datagen.flow_from_dataframe(dataframe=train_df1,
                                                  directory =PLANET_KAGGLE_JPEG_DIR, 
                                                  x_col='image_name', y_col=columns, subset='validation', 
                                                  batch_size=32,seed=42, shuffle=True, 
                                                  class_mode='raw', target_size=(128,128))


Found 32384 validated image filenames.
Found 8095 validated image filenames.


In [13]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

In [14]:
gc.collect()

0

In [15]:
# Define the model
def cnn_model():
    model = Sequential()

    # Convolution layers
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten layer
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=17, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(lr=0.0002), 
                  loss='binary_crossentropy', 
                  metrics=['binary_accuracy'])
    
    return model

In [16]:
optimizer = Adam(learning_rate=0.01)
print(optimizer)

In [17]:
#initialize the model
model = cnn_model()

# Preview the model architecture
model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________

In [18]:
# define callbacks
earlystop = EarlyStopping(monitor='val_binary_accuracy', 
                          patience=3, verbose=1, 
                          mode='max', 
                          restore_best_weights=True)

In [19]:
gc.collect()

0

In [20]:
# Fit the model 
if exec_train == True:
    history = model.fit(x = train_generator, 
                    steps_per_epoch = step_train_size, 
                    validation_data = val_generator,
                    validation_steps = step_val_size,
                    epochs = 40, 
                    callbacks = [earlystop])

In [25]:
if exec_load == True:
    model.load_weights('C:/Users/janni/RomansCode/Rainforest ML Projekt/l4/checkpoint')

OSError: Unable to open file (file signature not found)

In [ ]:
gc.collect()

In [ ]:
# Plot the training and validation accuracy and loss curves
if exec_train == True:
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['training', 'validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['training', 'validation'], loc='upper left')

    plt.show()

In [ ]:
if exec_val == True:

    file_names_val = val_generator.filenames

    val_generator.reset()
    pred_val = model.predict(val_generator, steps = step_val_size, verbose = 1)

    Y_val = val_generator.labels
    Y_hat_val = pred_val

In [ ]:
def find_best_thresholds(Y_hat, Y):
    N_tags = Y.shape[1]
    best_threshs = [0.2] * N_tags
    resolution = 100
    for jdx in range(N_tags):
        best_score = 0
        #threshs = np.zeros_like(best_threshs)
        threshs = best_threshs.copy()
        for kdx in range(resolution):
            kdx /= resolution
            threshs[jdx] = kdx
            Y_hat_thresh = (Y_hat > threshs).astype(float)
            score = fbeta_score(Y, Y_hat_thresh, beta=2, average="samples")
            if score > best_score:
                best_score = score
                best_threshs[jdx] = kdx
    
    global_best_score = fbeta_score(Y, (Y_hat > best_threshs).astype(float), beta=2, average="samples")
    print(f"threshs: {best_threshs} -- best score: {global_best_score}")
    
    return best_threshs

In [ ]:
if exec_val == True:
    
    internal_test_datagen = ImageDataGenerator(rescale = 1./255., featurewise_std_normalization=True, featurewise_center=True)

    
    internal_test_df = labels_df.iloc[:8095][:].reset_index().drop('index', axis =1)
    
    internal_test_generator = internal_test_datagen.flow_from_dataframe(dataframe=internal_test_df,
                                                  directory =PLANET_KAGGLE_JPEG_DIR, 
                                                  x_col='image_name', y_col=columns, 
                                                  batch_size=32,seed=42, shuffle=False, 
                                                  class_mode='raw', target_size=(128,128))
    
    Y_hat_internal_test = model.predict(internal_test_generator, steps = step_val_size, verbose = 1)
    Y_internal_test = internal_test_df[columns]
    
    
    best_threshs = find_best_thresholds(Y_hat=Y_hat_internal_test, Y=Y_internal_test)

In [ ]:
best_threshs

# submit you fucking result and end this "durchstich"

In [ ]:
gc.collect()

In [ ]:
# PLANET_KAGGLE_ROOT
# PLANET_KAGGLE_JPEG_DIR
# PLANET_KAGGLE_TIF_DIR
# PLANET_KAGGLE_TIF_TEST_DIR
# PLANET_KAGGLE_JPG_TEST_DIR
# PLANET_KAGGLE_JPG_TEST_EXTRA_DIR
# PLANET_KAGGLE_LABEL_CSV
# PLANET_KAGGLE_SUBMISSION_CSV

In [ ]:
##adding .jpg extension to image name in the sample submission file
if exec_submit == True:
    sample_submission = pd.read_csv(PLANET_KAGGLE_SUBMISSION_CSV)
    sample_submission1 = sample_submission.copy()
    sample_submission1['image_name'] = sample_submission1['image_name'].apply(lambda x: '{}.jpg'.format(x))
    sample_submission1.head()

In [ ]:
# Divide the sample submission file into two splits,
# first test1_df contains the first 40669 images 
if exec_submit == True:
    test_df1 = sample_submission1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
    test_df1.head()

In [ ]:
#initialize imagedatagenerator for the test images and also rescaling
if exec_submit == True:
    test_datagen = ImageDataGenerator(rescale = 1/255., featurewise_std_normalization=True, featurewise_center=True)

    #creating a generator for the images found in the first test image files
    test_gen = test_datagen.flow_from_dataframe(dataframe=test_df1, 
                                                directory=PLANET_KAGGLE_JPG_TEST_DIR, 
                                                x_col="image_name", 
                                                y_col=None, 
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False, 
                                                class_mode=None, 
                                                target_size=(128,128))

    step_test_size1 = int(np.ceil(test_gen.samples/test_gen.batch_size))
    

In [ ]:
#first, we reset the test generator to avoid shuffling of index 
if exec_submit == True:
    test_gen.reset()
    pred = model.predict(test_gen, steps=step_test_size1, verbose=1)

In [ ]:
# Get the filenames in the generator using the attribute .filenames
if exec_submit == True:
    file_names = test_gen.filenames

    # Convert the predicted values to a dataframe and join two labels together if prob(occurrance of the label) > thresholds 
    pred_tags = pd.DataFrame(pred)
    if use_best_thresholds == True:
        pred_tags = pred_tags.apply(lambda x: ' '.join(np.array(label_list)[x > best_threshs]), axis = 1)
    else:
        pred_tags = pred_tags.apply(lambda x: ' '.join(np.array(label_list)[x > manual_threshs]), axis = 1)
        
    #then the result should look like this 
    result1 = pd.DataFrame({'image_name': file_names, 'tags': pred_tags})
    result1.head()

In [ ]:
#second batch of the test dataset
if exec_submit == True:
    additional_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
    additional_df.head()

In [ ]:
#creating a generator for the second batch of test image files
if exec_submit == True:
    test_gen1 = test_datagen.flow_from_dataframe(dataframe=additional_df, 
                                                    directory=PLANET_KAGGLE_JPG_TEST_EXTRA_DIR, 
                                                    x_col='image_name', 
                                                    y_col=None, 
                                                    batch_size=32, 
                                                    shuffle=False, 
                                                    class_mode=None, 
                                                    target_size=(128,128))

    step_test_size2 = int(np.ceil(test_gen1.samples/test_gen1.batch_size))

In [ ]:
#we reset the generator to avoid shuffling, then make prediction on the generator
if exec_submit == True:
    test_gen1.reset()
    pred1 = model.predict(test_gen1, steps = step_test_size2, verbose = 1)

In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
if exec_submit == True:
    file_names1 = test_gen1.filenames

    #convert the predicted values to a dataframe
    #join two labels together if the prob(occurrance of the label) > 0.5
    pred_tags1 = pd.DataFrame(pred1)
    if use_best_thresholds == True:
        pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(label_list)[x > best_threshs]), axis = 1)
    else:
        pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(label_list)[x > manual_threshs]), axis = 1)

    result2 = pd.DataFrame({'image_name': file_names1, 'tags': pred_tags1})
    result2.head()

In [ ]:
# Final result of the predicted tags for the test images,
# we need to concat the first and second results in 
#that order to avoid shuffling the index
if exec_submit == True:
    final_df = pd.concat([result1, result2])

    final_df = final_df.reset_index().drop('index', axis =1)

    print(final_df.shape)
    final_df.head()

In [ ]:
# Remove the .jpg extension from the image_name of the last_result 
if exec_submit == True:
    final_df['image_name'] = final_df['image_name'].apply(lambda x: x[:-4])
    final_df.head()

In [ ]:
# Finally, we save the result to a csv file using the .to_csv() 
# method and setting the index to false.
if exec_submit == True:
    final_df.to_csv('l4/submissionl4.csv', index = False)
    model.save_weights('l4/l4_my_checkpoint')

In [ ]:
# # Save the weights
# model.save_weights('l3_my_checkpoint')

# # Create a new model instance
# model = create_model()

# # Restore the weights
# model.load_weights('./checkpoints/my_checkpoint')

# # Evaluate the model
# loss, acc = model.evaluate(test_images, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# results: visualization & analysis

In [ ]:
# gc.collect()
# model = cnn_model()
# model.load_weights('l1_my_checkpoint')

# # # Evaluate the model to make sure you loaded the right one
# loss, acc = model.evaluate(val_generator, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
#visual1a
#I would like to look at the pictures that I got wromg and see the actual labels vs the predicted ones. 
#this plot shall display 17 pics in a vertical line. so one example per label. in the second colum you have the true labels
#and in the third line you have the predicted labels
if exec_visualize == True:
    file_names = val_generator.filenames
    # Convert the predicted values to a dataframe and join two labels together if prob(occurrance of the label) > alpha
    pred = model.predict(val_generator, steps=step_val_size, verbose=1)
    pred_tags = pd.DataFrame(pred)
    pred_tags = pred_tags.apply(lambda x: ' '.join(np.array(label_list)[x > 0.2]), axis = 1)

In [ ]:
if exec_visualize == True:
    #then the result should look like this 
    prediction_val = pd.DataFrame({'image_name': file_names, 'tags': pred_tags})
    
    #get a one hot encoded prediction table
    for label in label_list:
        prediction_val[label] = prediction_val['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
    # Display head
    #here we have the truth table. note that we have val+train combined here
    truth_val = labels_df
    truth_val['image_name'] = truth_val['image_name'].apply(lambda x: '{}.jpg'.format(x))

In [ ]:
if exec_visualize == True:
    #per one hot label look for the rows where the pred does not match the truth
    label_to_print_index = 0
    label_to_print = label_list[label_to_print_index]
    
    #_x corresponds to the val set
    val_joined_pred=pd.merge(prediction_val,truth_val, on='image_name')
#     for label ined_pred[label+'correct'] = val_joined_pred[label+'_x'].apply(lambda x: 1 if x == val_joined_pred[label+'_y'] else 0)
 
    for label in label_list:
        val_joined_pred[label+'_correct'] = -1
        for index, row in val_joined_pred.iterrows():
            if row[label+'_x'] == row[label+'_y']:
                val_joined_pred.loc[index,label+'_correct'] = 1
            else:
                val_joined_pred.loc[index,label+'_correct'] = 0
                
  #df_wrong_prediction


In [ ]:
if exec_visualize == True:
    #extract randomly one pic from this list
    list_sampled_true_pos = []  
    list_sampled_false_pos = []
    list_sampled_true_neg  = []
    list_sampled_false_neg = []
    

    for label in label_list:

        #get the true positives
        temp_dict = {}
        temp_dict['chosen_label'] = label
        df_sampled_temp = val_joined_pred[(val_joined_pred[label+'_correct'] ==1) & (val_joined_pred[label+'_y'] ==1)]
        if not df_sampled_temp.empty:
            df_sampled_temp = df_sampled_temp.sample(1, replace=True)
            temp_dict['image_name'] = list(df_sampled_temp['image_name'])[0]
            temp_dict['true_labels'] = list(df_sampled_temp['tags_y'])
            temp_dict['predicted_labels'] = list(df_sampled_temp['tags_x'])
        else:
            temp_dict['image_name'] = "empty"
            temp_dict['true_labels'] = "empty"
            temp_dict['predicted_labels'] = "empty"
        list_sampled_true_pos.append(temp_dict)
        
        #get the false positives
        temp_dict = {}
        temp_dict['chosen_label'] = label
        df_sampled_temp = val_joined_pred[(val_joined_pred[label+'_correct'] ==0) & (val_joined_pred[label+'_y'] ==0)]
        if not df_sampled_temp.empty:
            df_sampled_temp = df_sampled_temp.sample(1, replace=True)
            temp_dict['image_name'] = list(df_sampled_temp['image_name'])[0]
            temp_dict['true_labels'] = list(df_sampled_temp['tags_y'])
            temp_dict['predicted_labels'] = list(df_sampled_temp['tags_x'])
        else:
            temp_dict['image_name'] = "empty"
            temp_dict['true_labels'] = "empty"
            temp_dict['predicted_labels'] = "empty"
        list_sampled_false_pos.append(temp_dict)
        
        #get the true negatives
        temp_dict = {}
        temp_dict['chosen_label'] = label
        df_sampled_temp = val_joined_pred[(val_joined_pred[label+'_correct'] ==1) & (val_joined_pred[label+'_y'] ==0)]
        if not df_sampled_temp.empty:
            df_sampled_temp = df_sampled_temp.sample(1, replace=True)
            temp_dict['image_name'] = list(df_sampled_temp['image_name'])[0]
            temp_dict['true_labels'] = list(df_sampled_temp['tags_y'])
            temp_dict['predicted_labels'] = list(df_sampled_temp['tags_x'])
        else:
            temp_dict['image_name'] = "empty"
            temp_dict['true_labels'] = "empty"
            temp_dict['predicted_labels'] = "empty"
        list_sampled_true_neg.append(temp_dict)
        
        #get the false negatives
        temp_dict = {}
        temp_dict['chosen_label'] = label
        df_sampled_temp = val_joined_pred[(val_joined_pred[label+'_correct'] ==0) & (val_joined_pred[label+'_y'] ==1)]
        if not df_sampled_temp.empty:
            df_sampled_temp = df_sampled_temp.sample(1, replace=True)
            temp_dict['image_name'] = list(df_sampled_temp['image_name'])[0]
            temp_dict['true_labels'] = list(df_sampled_temp['tags_y'])
            temp_dict['predicted_labels'] = list(df_sampled_temp['tags_x'])
        else:
            temp_dict['image_name'] = "empty"
            temp_dict['true_labels'] = "empty"
            temp_dict['predicted_labels'] = "empty"
        list_sampled_false_neg.append(temp_dict)



In [ ]:
def load_image(filename):
    '''Look through the directory tree to find the image you specified
    (e.g. train_10.tif vs. train_10.jpg)'''
    for dirname in os.listdir(PLANET_KAGGLE_ROOT):
        path = os.path.abspath(os.path.join(PLANET_KAGGLE_ROOT, dirname, filename))
        if os.path.exists(path):
            print('Found image {}'.format(path))
            return io.imread(path)
    # if you reach this line, you didn't find the image you're looking for
    print('Load failed: could not find image {}'.format(path))

In [ ]:
gc.collect()

In [ ]:
if exec_visualize == True:
    
    fig = plt.figure()
    fig.set_size_inches(30, 100)
    fig.suptitle('confusion matric pictures per target label', fontsize=16)
    for i, sample in enumerate(list_sampled_true_pos):
        a = fig.add_subplot(17, 4, i*4 +1)
        a.set_title("true_pos:" + sample['chosen_label'] + "\n"+ "true:"+str(sample['true_labels']) + "\n"+ "predicted:"+ str(sample['predicted_labels']))
        if sample['image_name'] != "empty":
            plt.imshow(load_image(sample['image_name']))
        else:
            plt.imshow(load_image('train_blanc.jpg'))
    for i, sample in enumerate(list_sampled_false_pos):
        a = fig.add_subplot(17, 4, i*4+2)
        a.set_title("false_pos:" + sample['chosen_label'] + "\n"+ "true:"+str(sample['true_labels']) + "\n"+ "predicted:"+ str(sample['predicted_labels']))
        if sample['image_name'] != "empty":
            plt.imshow(load_image(sample['image_name']))
        else:
            plt.imshow(load_image('train_blanc.jpg'))
    for i, sample in enumerate(list_sampled_true_neg):
        a = fig.add_subplot(17, 4, i*4+3)
        a.set_title("true_neg:" + sample['chosen_label'] + "\n"+ "true:"+str(sample['true_labels']) + "\n"+ "predicted:"+ str(sample['predicted_labels']))
        if sample['image_name'] != "empty":
            plt.imshow(load_image(sample['image_name']))
        else:
            plt.imshow(load_image('train_blanc.jpg'))
    for i, sample in enumerate(list_sampled_false_neg):
        a = fig.add_subplot(17, 4, i*4+4)
        a.set_title("false_neg:" + sample['chosen_label'] + "\n"+ "true:"+str(sample['true_labels']) + "\n"+ "predicted:"+ str(sample['predicted_labels']))
        if sample['image_name'] != "empty":
            plt.imshow(load_image(sample['image_name']))
        else:
            plt.imshow(load_image('train_blanc.jpg'))
    plt.savefig("l4/predictions_l4_rgb.jpg",  dpi='figure', format="jpg")

In [ ]:
#visual1b
#now it would be nice to see the false negatives in their r g b n slices


if exec_visualize == True:
    
    fig = plt.figure()
    fig.set_size_inches(30, 100)
    fig.suptitle('false negatives per target label', fontsize=16)
    for i, sample in enumerate(list_sampled_false_neg):
        
        if sample['image_name'] != "empty":
            bgrn_image = load_image(sample['image_name'].replace(".jpg", ".tif"))
            b, g, r, nir = bgrn_image[:, :, 0], bgrn_image[:, :, 1], bgrn_image[:, :, 2], bgrn_image[:, :, 3]
        else:
            b= load_image('train_blanc.jpg')
            g = b
            r = b
            nir = b
        
        a = fig.add_subplot(17, 4, i*4 +1)
        a.set_title("b:" + sample['chosen_label'] + "\n"+ "true:"+str(sample['true_labels']) + "\n"+ "predicted:"+ str(sample['predicted_labels']))
        plt.imshow(b)
            
        a = fig.add_subplot(17, 4, i*4+2)
        a.set_title("g:" + sample['chosen_label'] )
        plt.imshow(g)
            
        a = fig.add_subplot(17, 4, i*4+3)
        a.set_title("r:" + sample['chosen_label'] )
        plt.imshow(r)

        a = fig.add_subplot(17, 4, i*4+4)
        a.set_title("nir:" + sample['chosen_label'] )
        plt.imshow(nir)
            
    plt.savefig("l4/predictions_l4_rgbn.jpg",  dpi='figure', format="jpg")

In [ ]:
#visual5
#I would like to have the average of the prodiction probs plottet per label for both cases: Y_hat =1 and Y_hat =0

if exec_visualize == True:

    #y-hat is alwyas the predicted not the true value
    train_results_val = val_generator.labels
    train_results_val_hat = pred

    Y_val = np.array(train_results_val)
    Y_hat_val = np.array(train_results_val_hat)

    label_dict = {}
    for i, label in enumerate(label_list):
        label_dict[label] = i
        
    
    pos_probas, neg_probas = [], []
    for class_, idx in label_dict.items():
        pos_probas.append(Y_hat_val[np.where(Y_val[:, idx] != 0), idx].mean())
        neg_probas.append(Y_hat_val[np.where(Y_val[:, idx] == 0), idx].mean())
    go.Figure([
        go.Bar(x=list(label_dict), y=pos_probas, name="Y_hat proba | Y = 1"),
        go.Bar(x=list(label_dict), y=neg_probas, name="Y_hat proba | Y = 0")
    ]).show()

In [ ]:
#visual6
#I would like tos see how many labels were predicted

if exec_visualize == True:
    prediction_val[label_list].sum().sort_values().plot.bar()

In [ ]:
#visual4
#I would like to see the coooccurence matrices per label group and have them compared to the one of the training data
if exec_visualize == True:
    def make_cooccurence_matrix(label_list):
        numeric_df = prediction_val[label_list]; 
        c_matrix = numeric_df.T.dot(numeric_df)
        sns.heatmap(c_matrix)
        return c_matrix

    weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
    land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation']
    rare_labels = [l for l in label_list if prediction_val[label_list].sum()[l] < 1000]
    
    # Compute the co-ocurrence matrix
    make_cooccurence_matrix(label_list)

In [ ]:
if exec_visualize == True:

    
    # Compute the co-ocurrence matrix
    make_cooccurence_matrix(weather_labels)

In [ ]:
if exec_visualize == True:

    
    # Compute the co-ocurrence matrix
    make_cooccurence_matrix(land_labels)

In [ ]:
if exec_visualize == True:

    
    # Compute the co-ocurrence matrix
    make_cooccurence_matrix(rare_labels)

In [ ]:
#visual2
#I would like to see a diagram that shows the success percentages per label plottet against the amoint of training data per label

if exec_visualize == True:
    
    Y_val = np.array(train_results_val)
    Y_hat_val_binary = np.array(prediction_val[label_list])
    
    label_prediction_success = np.multiply(Y_val,Y_hat_val_binary)
    nr_rows = label_prediction_success.shape[0]
    
    nr_successes, nr_true, nr_successes_percent = [], [], []
    for class_, idx in label_dict.items():
        nr_successes.append(label_prediction_success[:, idx].sum())
        nr_true.append(Y_val[:, idx].sum())
        nr_successes_percent.append((label_prediction_success[:, idx].sum())/(Y_val[:, idx].sum()))
    go.Figure([
        go.Bar(x=list(label_dict), y=nr_successes, name="success"),
        go.Bar(x=list(label_dict), y=nr_true, name="true")
#         go.Bar(x=list(label_dict), y=nr_successes_percent, name="success rate")
    ]).show()

In [ ]:
if exec_visualize == True:
    #visual3
    #I would like to see the confusion matrices per label
    fig = make_subplots(cols=5, rows=4)
    for jdx in range(Y_val.shape[1]):
        y_val = Y_val[:, jdx].ravel()
        y_hat_val = (Y_hat_val[:, jdx].ravel() > 0.2).astype(float)
        tn, fp, fn, tp = confusion_matrix(y_val, y_hat_val).ravel()
        mat = np.array([[fn, tn], [tp, fp]])
        col = jdx // 4+1
        row = jdx % 4+1
        fig.add_trace(
            go.Heatmap(
                z=mat, text=[[f"fn: {fn}", f"tn: {tn}"], [f"tp: {tp}", f"fp: {fp}"]], 
                texttemplate="%{text}", colorscale='Viridis', name=list(label_dict.keys())[jdx],
                showscale=False
            ),
            col=col, row=row, 
        )
        fig.update_xaxes(title=list(label_dict.keys())[jdx], showticklabels=False, row=row, col=col)
        fig.update_yaxes(showticklabels=False, row=row, col=col)


    fig.update_layout(
        width=1200, height=800, title="Confusion matrices", 
    )
    fig.show()